<a href="https://colab.research.google.com/github/DelMashiry-dev/DelMashiry-dev/blob/main/REALTIMEPREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount the Drive Containing the folder with the dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Medical_Resource_Prediction

In [ ]:
import pandas as pd
# Read first 100000 rows but only specific columns
df = pd.read_csv('/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv', nrows=100000, usecols=['iso_code', 'continent', 'location', 'date','total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'reproduction_rate',
    'icu_patients', 'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions','total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
    'total_tests', 'new_tests', 'positive_rate','population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older','cardiovasc_death_rate', 'diabetes_prevalence'])
df.fillna(0, inplace=True)
from IPython.display import display
display(df.head(5000))

# Identify Relevant Features



In [ ]:
relevant_columns = [
    'iso_code', 'continent', 'location', 'date','total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'reproduction_rate',
    'icu_patients', 'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions','total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
    'total_tests', 'new_tests', 'positive_rate', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
    'hospital_beds_per_thousand', 'cardiovasc_death_rate', 'diabetes_prevalence'
]

# Data Cleaning
# Remove Irrelevant Columns
# Filter the dataset to keep only the selected columns:



In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv')

# Keep relevant columns
df = df[relevant_columns]
from IPython.display import display
display(df.head(5))

# Handle Missing Values



#Imputation Strategy



In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows with missing iso_code, location, or date
df = df.dropna(subset=['iso_code', 'location', 'date'])

# Time-series imputation for disease and healthcare metrics
time_series_cols = [
    'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'reproduction_rate',
    'icu_patients', 'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
    'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
    'total_tests', 'new_tests', 'positive_rate'
]
for col in time_series_cols:
    df[col] = df.groupby('location')[col].fillna(method='ffill').interpolate()

# Zero imputation for ICU/hospital metrics if still missing (early outbreak)
zero_impute_cols = ['icu_patients', 'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions']
df[zero_impute_cols] = df[zero_impute_cols].fillna(0)

# Median imputation for demographic/healthcare capacity features
median_impute_cols = [
    'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
    'hospital_beds_per_thousand', 'cardiovasc_death_rate', 'diabetes_prevalence'
]
for col in median_impute_cols:
    df[col] = df.groupby('continent')[col].transform(lambda x: x.fillna(x.median()))

# Handle remaining missing values (if any) with global median
for col in median_impute_cols:
    df[col] = df[col].fillna(df[col].median())

    display(df.head(5000))

# Check for Outliers



In [ ]:
for col in time_series_cols:
    df[col] = df.groupby('location')[col].transform(lambda x: x.clip(upper=x.quantile(0.99)))

# Feature Engineering



In [ ]:
# Lag features
for col in ['new_cases', 'icu_patients', 'hosp_patients']:
    df[f'{col}_lag7'] = df.groupby('location')[col].shift(7)
    df[f'{col}_lag14'] = df.groupby('location')[col].shift(14)

# Rolling averages
df['new_cases_7d_avg'] = df.groupby('location')['new_cases'].rolling(7, min_periods=1).mean().reset_index(level=0, drop=True)
df['new_deaths_7d_avg'] = df.groupby('location')['new_deaths'].rolling(7, min_periods=1).mean().reset_index(level=0, drop=True)

# Proxy for ventilator demand (e.g., 50% of ICU patients)
df['ventilator_demand'] = df['icu_patients'] * 0.5

# Final Dataset
# Ensure the dataset is sorted by location and date for time-series modeling:



In [ ]:
df = df.sort_values(['location', 'date'])

In [ ]:
df.info()

# Time Series Plot Example (Plotly)



# This creates an interactive line plot where you can hover over points to see exact values, zoom in on specific time periods, and compare trends across locations. This is particularly useful for understanding temporal patterns in disease progression and resource demand



In [ ]:
import plotly.express as px

# Plot multiple metrics over time, colored by location
fig = px.line(df, x='date', y=['total_cases', 'new_cases', 'icu_patients'], color='location', title='Key Metrics Over Time')
fig.show()

# Scatter Plot Example (Seaborn)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot total_cases vs. icu_patients, colored by location
sns.scatterplot(data=df, x='total_cases', y='icu_patients', hue='location')
plt.title('Total Cases vs. ICU Patients')
plt.show()

# This scatter plot helps visualize the relationship between disease spread and ICU demand, which is a proxy for ventilator and medical personnel needs. You can extend this to plot total_vaccinations vs. new_cases to assess vaccination impact.



# Correlation Heatmap (Seaborn)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select numeric columns for correlation
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Compute and plot correlation heatmap
plt.figure(figsize=(40, 38))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

#This heatmap shows correlations between all numeric features, helping identify which factors (e.g., population_density, median_age) are most related to resource demand



# Bar Chart(Plotly)



In [ ]:
import plotly.express as px

# Get the latest data for each location
latest_df = df[df['date'] == df['date'].max()]

# Create bar chart for latest total_cases
fig = px.bar(latest_df, x='location', y='total_cases', title='Latest Total Cases by Location')
fig.show()

#This bar chart compares the most recent total cases across locations, which can help identify regions with high demand for real-time monitoring



In [ ]:
import plotly.express as px

# Get the latest data for each location
latest_df = df[df['date'] == df['date'].max()]

# Sort by total cases in descending order
latest_df = latest_df.sort_values(by='total_cases', ascending=False)

# Create a horizontal bar chart with color by continent
fig = px.bar(latest_df, x='total_cases', y='location', color='continent', orientation='h',
             title='Latest Total Cases by Location, Colored by Continent')
fig.show()

# Geographic Map (Plotly)



In [ ]:
import plotly.express as px

# Create choropleth map for total_cases by country, animated over time
fig = px.choropleth(df, locations='iso_code', locationmode='ISO-3', color='total_cases', hover_name='location', animation_frame='date', title='Total Cases Over Time by Country')
fig.show()

# This animated map shows the spatial distribution of total cases over time, which is useful for understanding regional disparities and can be sourced from real-time APIs like OWID



In [ ]:
import pandas as pd
from IPython.display import display

# Assuming your cleaned DataFrame is named 'df_clean'
# If not, replace with your actual cleaned DataFrame name

def display_final_dataset(df, num_rows=5, all_columns=False):
    """
    Displays the final prepared dataset in a clean table format

    Parameters:
    - df: Your cleaned DataFrame
    - num_rows: Number of rows to display (default: 5)
    - all_columns: Whether to show all columns (default: False for truncated view)
    """

    # Configure display options
    pd.set_option('display.max_columns', None if all_columns else 10)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', 20)

    # Create a styled table
    styled_df = (df.head(num_rows)
                 .style
                 .set_properties(**{'text-align': 'center'})
                 .set_table_styles([{
                     'selector': 'th',
                     'props': [('background-color', '#40466e'),
                              ('color', 'white'),
                              ('font-weight', 'bold')]
                 }])
                 .background_gradient(cmap='Blues', subset=df.select_dtypes(include='number').columns)
                 .format(None, na_rep="NA"))

    # Display in notebook
    display(styled_df)

    # Show dataset info
    print("\n\033[1mDataset Summary:\033[0m")
    print(f"Total Rows: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print("\n\033[1mColumn Types:\033[0m")
    print(df.dtypes.value_counts())

    # Show NA counts if any exist
    if df.isna().sum().sum() > 0:
        print("\n\033[1mMissing Values:\033[0m")
        missing = df.isna().sum()[df.isna().sum() > 0]
        print(missing)
    else:
        print("\n\033[1mNo missing values found!\033[0m")

# Usage - call the function with your cleaned DataFrame
display_final_dataset(df, num_rows=5, all_columns=True)

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

def display_and_export_dataset(df, num_rows=5, export_name="cleaned_dataset"):
    """
    Displays the final dataset with enhanced formatting and export options

    Parameters:
    - df: Your cleaned DataFrame
    - num_rows: Number of rows to display
    - export_name: Base name for exported files
    """

    # ==============================================
    # 1. SPECIAL COLUMN FORMATTING
    # ==============================================
    format_rules = {
        # Medical resource columns (integers with comma separators)
        r'(beds|patients|ventilators|staff|ppe)': '{:,.0f}',

        # Percentage columns (show as % with 1 decimal)
        r'(rate|ratio|percent|per_hundred)': '{:.1%}',

        # Date columns (standard date format)
        'date': '{:%Y-%m-%d}',

        # Small decimal numbers (3 decimal places)
        r'(growth|factor|index)': '{:.3f}'
    }

    # ==============================================
    # 2. CREATE STYLED TABLE
    # ==============================================
    styler = (df.head(num_rows)
              .style
              .set_properties(**{'text-align': 'center'})
              .set_table_styles([{
                  'selector': 'th',
                  'props': [
                      ('background-color', '#2a3f5f'),
                      ('color', 'white'),
                      ('font-weight', 'bold'),
                      ('position', 'sticky'),
                      ('top', '0')
                  ]
              }]))

    # Apply special formatting
    for regex, formatter in format_rules.items():
        cols = df.filter(regex=regex, axis=1).columns
        if not cols.empty:
            styler.format(formatter, subset=cols)

    # Highlight important metrics
    medical_cols = df.filter(regex='icu|hosp|ventilator|ppe').columns
    if not medical_cols.empty:
        styler.background_gradient(
            cmap='YlOrRd',
            subset=medical_cols,
            vmin=0, vmax=df[medical_cols].max().max()
        )

    # ==============================================
    # 3. DISPLAY RESULTS
    # ==============================================
    print("="*80)
    print(f"\033[1m{'CLEANED DATASET PREVIEW':^80}\033[0m")
    print("="*80)
    display(styler)

    # ==============================================
    # 4. STATISTICAL SUMMARIES
    # ==============================================
    print("\n\033[1mSTATISTICAL SUMMARIES\033[0m")
    print("-"*80)

    # Numeric columns summary
    numeric_df = df.select_dtypes(include=np.number)
    if not numeric_df.empty:
        print("\n\033[4mNumeric Columns:\033[0m")
        display(numeric_df.describe().style.format("{:.2f}"))

    # Categorical columns summary
    categorical_df = df.select_dtypes(include='object')
    if not categorical_df.empty:
        print("\n\033[4mCategorical Columns:\033[0m")
        for col in categorical_df.columns:
            print(f"\n• {col}:")
            print(df[col].value_counts(dropna=False).head())

    # ==============================================
    # 5. EXPORT OPTIONS
    # ==============================================
    print("\n\033[1mEXPORT OPTIONS\033[0m")
    print("-"*80)

    try:
        # Excel Export
        excel_file = f"{export_name}.xlsx"
        with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='Data', index=False)

            # Add summary sheets
            numeric_df.describe().to_excel(writer, sheet_name='Numeric Summary')
            if not categorical_df.empty:
                pd.concat([
                    df[col].value_counts(dropna=False).rename(col)
                    for col in categorical_df.columns
                ], axis=1).to_excel(writer, sheet_name='Category Counts')

            # Get workbook objects
            workbook = writer.book
            worksheet = writer.sheets['Data']

            # Add Excel formatting
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'top',
                'fg_color': '#2a3f5f',
                'font_color': 'white',
                'border': 1
            })

            # Apply header format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)

            # Auto-adjust column widths
            for i, col in enumerate(df.columns):
                max_len = max((
                    df[col].astype(str).map(len).max(),  # Data length
                    len(str(col))  # Header length
                )) + 2
                worksheet.set_column(i, i, min(max_len, 50))

        print(f"✓ Excel file saved as: {excel_file}")

        # HTML Export
        html_file = f"{export_name}.html"
        styler.to_html(html_file)
        print(f"✓ HTML file saved as: {html_file}")

    except Exception as e:
        print(f"Export failed: {str(e)}")

# Usage example:
# display_and_export_dataset(df_clean, num_rows=10, export_name="medical_resources")

In [ ]:
display_and_export_dataset(df,
                         num_rows=10,
                         export_name="my_cleaned_data")

In [ ]:
!pip install xlsxwriter


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

def display_and_export_dataset(df, num_rows=5, export_name="cleaned_dataset"):
    """
    Displays the final dataset with enhanced formatting and export options

    Parameters:
    - df: Your cleaned DataFrame
    - num_rows: Number of rows to display
    - export_name: Base name for exported files
    """

    # ==============================================
    # 1. SPECIAL COLUMN FORMATTING
    # ==============================================
    format_rules = {
        # Medical resource columns
        r'(beds|patients|ventilators|staff|ppe)': '{:,.0f}',
        # Percentage columns
        r'(rate|ratio|percent|per_hundred)': '{:.1%}',
        # Date columns
        'date': '{:%Y-%m-%d}',
        # Small decimal numbers
        r'(growth|factor|index)': '{:.3f}'
    }

    # ==============================================
    # 2. CREATE STYLED TABLE
    # ==============================================
    styler = (df.head(num_rows)
              .style
              .set_properties(**{'text-align': 'center'})
              .set_table_styles([{
                  'selector': 'th',
                  'props': [
                      ('background-color', '#2a3f5f'),
                      ('color', 'white'),
                      ('font-weight', 'bold')
                  ]
              }]))

    # Apply formatting
    for regex, formatter in format_rules.items():
        cols = df.filter(regex=regex, axis=1).columns
        if not cols.empty:
            styler.format(formatter, subset=cols)

    # Highlight medical metrics
    medical_cols = df.filter(regex='icu|hosp|ventilator|ppe').columns
    if not medical_cols.empty:
        styler.background_gradient(
            cmap='YlOrRd',
            subset=medical_cols,
            vmin=0, vmax=df[medical_cols].max().max()
        )

    # ==============================================
    # 3. DISPLAY RESULTS
    # ==============================================
    print("="*80)
    print(f"\033[1m{'CLEANED DATASET PREVIEW':^80}\033[0m")
    print("="*80)
    display(styler)

    # ==============================================
    # 4. STATISTICAL SUMMARIES
    # ==============================================
    print("\n\033[1mSTATISTICAL SUMMARIES\033[0m")
    print("-"*80)

    # Numeric summary
    numeric_df = df.select_dtypes(include=np.number)
    if not numeric_df.empty:
        print("\n\033[4mNumeric Columns:\033[0m")
        display(numeric_df.describe().style.format("{:.2f}"))

    # Categorical summary
    categorical_df = df.select_dtypes(include='object')
    if not categorical_df.empty:
        print("\n\033[4mCategorical Columns:\033[0m")
        for col in categorical_df.columns:
            print(f"\n• {col}:")
            print(df[col].value_counts(dropna=False).head())

    # ==============================================
    # 5. EXPORT OPTIONS
    # ==============================================
    print("\n\033[1mEXPORT OPTIONS\033[0m")
    print("-"*80)

    # HTML Export (always available)
    html_file = f"{export_name}.html"
    styler.to_html(html_file)
    print(f"✓ HTML file saved as: {html_file}")

    # Excel Export (only if xlsxwriter is available)
    try:
        import xlsxwriter
        excel_file = f"{export_name}.xlsx"
        with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='Data', index=False)

            # Add summary sheets
            numeric_df.describe().to_excel(writer, sheet_name='Numeric Summary')
            if not categorical_df.empty:
                pd.concat([
                    df[col].value_counts(dropna=False).rename(col)
                    for col in categorical_df.columns
                ], axis=1).to_excel(writer, sheet_name='Category Counts')

            # Formatting
            workbook = writer.book
            worksheet = writer.sheets['Data']
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'fg_color': '#2a3f5f',
                'font_color': 'white',
                'border': 1
            })

            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)

            # Auto-adjust columns
            for i, col in enumerate(df.columns):
                max_len = max((df[col].astype(str).map(len).max(), len(str(col)))) + 2
                worksheet.set_column(i, i, min(max_len, 50))

        print(f"✓ Excel file saved as: {excel_file}")

    except ImportError:
        print("ℹ️ Excel export requires xlsxwriter. Install with: !pip install xlsxwriter")
    except Exception as e:
        print(f"⚠️ Excel export failed: {str(e)}")

# Usage example
# display_and_export_dataset(df_clean, num_rows=10, export_name="medical_data")